# 9장 CNN을 활용한 이미지 인식

* "부록3 매트플롯립 입문"에서 한글 폰트를 올바르게 출력하기 위한 설치 방법을 설명했다. 설치 방법은 다음과 같다.

In [ ]:
!sudo apt-get install -y fonts-nanum* | tail -n 1
!sudo fc-cache -fv
!rm -rf ~/.cache/matplotlib

In [ ]:
# 필요 라이브러리 설치

!pip install torchviz | tail -n 1
!pip install torchinfo | tail -n 1

* 모든 설치가 끝나면 한글 폰트를 바르게 출력하기 위해 **[런타임]** -> **[런타임 다시시작]**을 클릭한 다음, 아래 셀부터 코드를 실행해 주십시오.

In [ ]:
# 라이브러리 임포트

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# 폰트 관련 용도
import matplotlib.font_manager as fm

# 나눔 고딕 폰트의 경로 명시
path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=path, size=10).get_name()

In [ ]:
# 파이토치 관련 라이브러리

import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchviz import make_dot
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
# warning 표시 끄기
import warnings
warnings.simplefilter('ignore')

# 기본 폰트 설정
plt.rcParams['font.family'] = font_name

# 기본 폰트 사이즈 변경
plt.rcParams['font.size'] = 14

# 기본 그래프 사이즈 변경
plt.rcParams['figure.figsize'] = (6,6)

# 기본 그리드 표시
# 필요에 따라 설정할 때는, plt.grid()
plt.rcParams['axes.grid'] = True

# 마이너스 기호 정상 출력
plt.rcParams['axes.unicode_minus'] = False

# 넘파이 부동소수점 자릿수 표시
np.set_printoptions(suppress=True, precision=4)

### GPU 확인하기

In [ ]:
# 디바이스 할당

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

## 9.3 CNN의 처리 개요

In [ ]:
data_root = './data'

# 샘플 손글씨 숫자 데이터 가져오기
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_set = datasets.MNIST(
    root = data_root,  train = True,
    download = True, transform = transform)

image, label = train_set[0]
image = image.view(1,1,28,28)

In [ ]:
image.shape

In [ ]:
image[0]

In [ ]:
# 대각선상에만 가중치를 갖는 특수한 합성곱 함수를 만듦
conv1 = nn.Conv2d(1, 1, 3) # 입력채널 : 1, 출력채널 : 1, 커널 크기 : 3x3

# bias를 0으로
nn.init.constant_(conv1.bias, 0.0)

# weight를 특수한 값으로
w1_np = np.array([[0,0,1],[0,1,0],[1,0,0]])
w1 = torch.tensor(w1_np).float()
w1 = w1.view(1,1,3,3)
conv1.weight.data = w1

In [ ]:
train_set[0]

In [ ]:
# 손글씨 숫자에 3번 합성곱 처리를 함
image, label = train_set[0]
image = image.view(1,1,28,28)
w1 = conv1(image)
w2 = conv1(w1)
w3 = conv1(w2)
images = [image, w1, w2, w3]

In [ ]:
images

In [ ]:
images[0].data.numpy()

In [ ]:
# 결과 화면 출력

plt.figure(figsize=(5, 1))
for i in range(4):
    size = 28 - i*2
    ax = plt.subplot(1, 4, i+1)
    img = images[i].data.numpy()
    plt.imshow(img.reshape(size, size),cmap='gray_r')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## 9.4 파이토치에서 CNN을 구현하는 방법

### nn.Conv2d 와 nn.MaxPool2d

In [ ]:
# CNN 모델 전반 부분, 레이어 함수 정의
conv1 = nn.Conv2d(3, 32, 3)          # 입력 채널: 3, 출력 채널: 32, 커널 크기: 3x3
relu = nn.ReLU(inplace=True)       # 활성화 함수 ReLU
conv2 = nn.Conv2d(32, 32, 3)         # 입력 채널: 32, 출력 채널: 32, 커널 크기: 3x3
maxpool = nn.MaxPool2d((2, 2))       # 풀링 영역 크기: 2x2

In [ ]:
# conv1 확인
print(conv1)
# conv1 내부 변수의 shape 확인
print(f"conv1 weight shape: {conv1.weight.shape}")
print(f"conv1 bias shape: {conv1.bias.shape}")

# conv2 내부 변수의 shape 확인
print(f"conv2 weight shape: {conv2.weight.shape}")
print(f"conv2 bias shape: {conv2.bias.shape}")

In [ ]:
# conv1의 weight[0]는 0번째 출력 채널의 가중치
w = conv1.weight[0]

# weight[0]의 shape과 값 확인
print(w.shape)
print(w.data.numpy())

In [ ]:
# 더미로 입력과 같은 사이즈를 갖는 텐서를 생성
inputs = torch.randn(100, 3, 32, 32)
print(inputs.shape)

In [ ]:
# CNN 전반부 처리 시뮬레이션

x1 = conv1(inputs)
x2 = relu(x1)
x3 = conv2(x2)
x4 = relu(x3)
x5 = maxpool(x4)

In [ ]:
# 각 변수의 shape 확인

print(f"{'Input':<10}: {inputs.shape}")
print(f"{'conv1':<10}: {x1.shape}")
print(f"{'relu':<10}: {x2.shape}")
print(f"{'conv2':<10}: {x3.shape}")
print(f"{'relu':<10}: {x4.shape}")
print(f"{'maxpool':<10}: {x5.shape}")

### nn.Sequential

In [ ]:
# 함수 정의
features = nn.Sequential(
    conv1,
    relu,
    conv2,
    relu,
    maxpool
)

In [ ]:
# 동작 테스트
outputs = features(inputs)

# 결과 확인
print(outputs.shape)

### nn.Flatten

In [ ]:
# 함수 정의
flatten = nn.Flatten()

# 동작 테스트 (이전 단계의 outputs를 입력으로 사용)
outputs2 = flatten(outputs)

# 결과 확인
print(f"Flatten 이전 shape: {outputs.shape}")
print(f"Flatten 이후 shape: {outputs2.shape}")

In [ ]:
# 32 * 14 * 14
# 6272

## 9.5 공통 함수 사용하기

### eval_loss(손실 계산)

In [ ]:
# 손실 계산용
def eval_loss(loader, device, net, criterion):

    # 데이터로더에서 처음 한 개 세트를 가져옴
    for images, labels in loader:
        break

    # 디바이스 할당
    inputs = images.to(device)
    labels = labels.to(device)

    # 예측 계산
    outputs = net(inputs)

    # 손실 계산
    loss = criterion(outputs, labels)

    return loss

### fit(학습)

In [ ]:
# 학습용 함수
def fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history):

    # tqdm 라이브러리 임포트
    from tqdm.notebook import tqdm

    base_epochs = len(history)

    for epoch in range(base_epochs, num_epochs+base_epochs):
        train_loss = 0
        train_acc = 0
        val_loss = 0
        val_acc = 0

        # 훈련 페이즈
        net.train()
        count = 0

        for inputs, labels in tqdm(train_loader):
            count += len(labels)
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 경사 초기화
            optimizer.zero_grad()

            # 예측 계산
            outputs = net(inputs)

            # 손실 계산
            loss = criterion(outputs, labels)
            train_loss += loss.item()

            # 경사 계산
            loss.backward()

            # 파라미터 수정
            optimizer.step()

            # 예측 라벨 산출
            predicted = torch.max(outputs, 1)[1]

            # 정답 건수 산출
            train_acc += (predicted == labels).sum().item()

            # 손실과 정확도 계산
            avg_train_loss = train_loss / count
            avg_train_acc = train_acc / count

        # 예측 페이즈
        net.eval()
        count = 0

        for inputs, labels in test_loader:
            count += len(labels)
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 예측 계산
            outputs = net(inputs)

            # 손실 계산
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # 예측 라벨 산출
            predicted = torch.max(outputs, 1)[1]

            # 정답 건수 산출
            val_acc += (predicted == labels).sum().item()

            # 손실과 정확도 계산
            avg_val_loss = val_loss / count
            avg_val_acc = val_acc / count

        print (f'Epoch [{(epoch+1)}/{num_epochs+base_epochs}], loss: {avg_train_loss:.5f} acc: {avg_train_acc:.5f} val_loss: {avg_val_loss:.5f}, val_acc: {avg_val_acc:.5f}')
        item = np.array([epoch+1, avg_train_loss, avg_train_acc, avg_val_loss, avg_val_acc])
        history = np.vstack((history, item))
    return history

### eval_history(학습 로그)

In [ ]:
# 학습 로그 해석

def evaluate_history(history):
    # 손실과 정확도 확인
    print(f'초기상태 : 손실 : {history[0,3]:.5f}  정확도 : {history[0,4]:.5f}')
    print(f'최종상태 : 손실 : {history[-1,3]:.5f}  정확도 : {history[-1,4]:.5f}' )

    num_epochs = len(history)
    unit = num_epochs / 10

    # 학습 곡선 출력(손실)
    plt.figure(figsize=(9,8))
    plt.plot(history[:,0], history[:,1], 'b', label='훈련')
    plt.plot(history[:,0], history[:,3], 'k', label='검증')
    plt.xticks(np.arange(0,num_epochs+1, unit))
    plt.xlabel('반복 횟수')
    plt.ylabel('손실')
    plt.title('학습 곡선(손실)')
    plt.legend()
    plt.show()

    # 학습 곡선 출력(정확도)
    plt.figure(figsize=(9,8))
    plt.plot(history[:,0], history[:,2], 'b', label='훈련')
    plt.plot(history[:,0], history[:,4], 'k', label='검증')
    plt.xticks(np.arange(0,num_epochs+1,unit))
    plt.xlabel('반복 횟수')
    plt.ylabel('정확도')
    plt.title('학습 곡선(정확도)')
    plt.legend()
    plt.show()

### show_images_labels(예측 결과 표시)

In [ ]:
# 이미지와 라벨 표시
def show_images_labels(loader, classes, net, device):

    # 데이터로더에서 처음 1세트를 가져오기
    for images, labels in loader:
        break
    # 표시 수는 50개
    n_size = min(len(images), 50)

    if net is not None:
      # 디바이스 할당
      inputs = images.to(device)
      labels = labels.to(device)

      # 예측 계산
      outputs = net(inputs)
      predicted = torch.max(outputs,1)[1]
      #images = images.to('cpu')

    # 처음 n_size개 표시
    plt.figure(figsize=(20, 15))
    for i in range(n_size):
        ax = plt.subplot(5, 10, i + 1)
        label_name = classes[labels[i]]
        # net이 None이 아닌 경우는 예측 결과도 타이틀에 표시함
        if net is not None:
          predicted_name = classes[predicted[i]]
          # 정답인지 아닌지 색으로 구분함
          if label_name == predicted_name:
            c = 'k'
          else:
            c = 'b'
          ax.set_title(label_name + ':' + predicted_name, c=c, fontsize=20)
        # net이 None인 경우는 정답 라벨만 표시
        else:
          ax.set_title(label_name, fontsize=20)
        # 텐서를 넘파이로 변환
        image_np = images[i].numpy().copy()
        # 축의 순서 변경 (channel, row, column) -> (row, column, channel)
        img = np.transpose(image_np, (1, 2, 0))
        # 값의 범위를[-1, 1] -> [0, 1]로 되돌림
        img = (img + 1)/2
        # 결과 표시
        plt.imshow(img)
        ax.set_axis_off()
    plt.show()


### torch_seed(난수 초기화)

In [ ]:
# 파이토치 난수 고정

def torch_seed(seed=123):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True


## 9.6 데이터 준비

In [ ]:
# Transforms의 정의

# transform1: 1계 텐서화 (전결합형 신경망용)
transform1 = transforms.Compose([
    transforms.ToTensor(),                   # 데이터를 PyTorch 텐서로 변환
    transforms.Normalize(0.5, 0.5),          # 데이터를 -1 ~ 1 범위로 정규화
    transforms.Lambda(lambda x: x.view(-1)), # 텐서를 1차원으로 평탄화
])

# transform2: 정규화만 실시 (CNN용)
transform2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
])

In [ ]:
# 데이터 취득용 함수 datasets

data_root = './data'

# 훈련 데이터셋 (1계 텐서 버전)
train_set1 = datasets.CIFAR10(
    root = data_root, train = True,
    download = True, transform = transform1)

# 검증 데이터셋 (1계 텐서 버전)
test_set1 = datasets.CIFAR10(
    root = data_root, train = False,
    download = True, transform = transform1)

# 훈련 데이터셋 (3계 텐서 버전)
train_set2 = datasets.CIFAR10(
    root =  data_root, train = True,
    download = True, transform = transform2)

# 검증 데이터셋 (3계 텐서 버전)
test_set2 = datasets.CIFAR10(
    root = data_root, train = False,
    download = True, transform = transform2)

### 데이터셋 확인

In [ ]:
image1, label1 = train_set1[0]
image2, label2 = train_set2[0]
print(f"transform1 적용 후 shape: {image1.shape}")
print(f"transform2 적용 후 shape: {image2.shape}")

In [ ]:
3 * 32 * 32

In [ ]:
# 데이터로더 정의

# 미니 배치 사이즈 지정
batch_size = 100

# 훈련용 데이터로더
# 훈련용이므로 셔플을 True로 설정
train_loader1 = DataLoader(train_set1, batch_size=batch_size, shuffle=True)

# 검증용 데이터로더
# 검증용이므로 셔플하지 않음
test_loader1 = DataLoader(test_set1,  batch_size=batch_size, shuffle=False)

# 훈련용 데이터로더
# 훈련용이므로 셔플을 True로 설정
train_loader2 = DataLoader(train_set2, batch_size=batch_size, shuffle=True)

# 검증용 데이터로더
# 검증용이므로 셔플하지 않음
test_loader2 = DataLoader(test_set2,  batch_size=batch_size, shuffle=False)


In [ ]:
for images1, labels1 in train_loader1:
    break # 첫 번째 배치만 확인
for images2, labels2 in train_loader2:
    break # 첫 번째 배치만 확인

print(f"train_loader1의 배치 shape: {images1.shape}")
print(f"train_loader2의 배치 shape: {images2.shape}")

In [ ]:
# 정답 라벨 정의
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# 검증 데이터의 처음 50개를 출력
show_images_labels(test_loader2, classes, None, None)

## 9.7 모델 정의(전결합형)

### 학습용 파라미터 설정

In [ ]:
# 입력 차원수는 3*32*32=3072
n_input = image1.view(-1).shape[0]

# 출력 차원수
# 분류 클래스의 수이므로　10
n_output = len(set(list(labels1.data.numpy())))

# 은닉층의 노드수
n_hidden = 128

# 결과 확인
print(f'n_input: {n_input}  n_hidden: {n_hidden} n_output: {n_output}')

In [ ]:
image1

In [ ]:
# 모델 정의
# 3072입력 10출력 1은닉층을 포함한 신경망 모델

class Net(nn.Module):
    def __init__(self, n_input, n_output, n_hidden):
        super().__init__()

        # 은닉층 정의(은닉층의 노드수 : n_hidden)
        self.l1 = nn.Linear(n_input, n_hidden)

        # 출력층의 정의
        self.l2 = nn.Linear(n_hidden, n_output)

        # ReLU 함수 정의
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x1 = self.l1(x)
        x2 = self.relu(x1)
        x3 = self.l2(x2)
        return x3

### 모델 인스턴스 생성과 GPU 할당

In [ ]:
# 모델 인스턴스 생성
net = Net(n_input, n_output, n_hidden).to(device)

# 손실 함수： 교차 엔트로피 함수
criterion = nn.CrossEntropyLoss()

# 학습률
lr = 0.01

# 최적화 함수: 경사 하강법
optimizer = torch.optim.SGD(net.parameters(), lr=lr)

In [ ]:
# 모델 개요 표시 1

print(net)

In [ ]:
# 모델 개요 표시 2

summary(net, (100,3072),depth=1)

In [ ]:
# 손실 계산
loss = eval_loss(test_loader1, device, net, criterion)

# 손실 계산 그래프 시각화
g = make_dot(loss, params=dict(net.named_parameters()))
display(g)

## 9.8 결과(전결합형)

### 학습

In [ ]:
# 전결합형 모델의 초기화와 학습
torch_seed() # 난수 초기화
net = Net(n_input, n_hidden, n_output).to(device) # 모델 인스턴스 생성 및 디바이스 할당
criterion = nn.CrossEntropyLoss() # 손실 함수: 교차 엔트로피 함수
lr = 0.01 # 학습률
optimizer = optim.SGD(net.parameters(), lr=lr) # 최적화 함수: 경사 하강법
num_epochs = 50 # 반복 횟수
history = np.zeros((0, 5)) # 평가 결과 기록

# 학습
history = fit(net, optimizer, criterion, num_epochs, train_loader1, test_loader1, device, history)

### 평가

In [ ]:
# 평가

evaluate_history(history)

## 9.9 모델 정의(CNN)

In [ ]:
# CNN 모델 클래스 정의
class CNN(nn.Module):
    def __init__(self, n_output, n_hidden):
        super().__init__()
        # 레이어 정의
        self.conv1 = nn.Conv2d(3, 32, 3) # 입력 채널:3, 출력 채널:32, 커널:3x3
        self.conv2 = nn.Conv2d(32, 32, 3) # 입력 채널:32, 출력 채널:32, 커널:3x3
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d((2,2))
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(6272, n_hidden) # 입력:6272, 출력:은닉노드 수
        self.l2 = nn.Linear(n_hidden, n_output) # 입력:은닉노드 수, 출력:클래스 수

        # 역할별로 그룹화
        self.features = nn.Sequential(
            self.conv1, self.relu, self.conv2, self.relu, self.maxpool
        )
        self.classifier = nn.Sequential(self.l1, self.relu, self.l2)

    # '조립 라인': 데이터 흐름 정의
    def forward(self, x):
        x1 = self.features(x)   # 특징 추출
        x2 = self.flatten(x1)   # 1차원으로 펼치기
        x3 = self.classifier(x2) # 최종 분류
        return x3

### 모델 인스턴스 생성

In [ ]:
# 모델 인스턴스 생성
net = CNN(n_output, n_hidden).to(device)

# 손실 함수： 교차 엔트로피 함수
criterion = nn.CrossEntropyLoss()

# 학습률
lr = 0.01

# 최적화 함수: 경사 하강법
optimizer = torch.optim.SGD(net.parameters(), lr=lr)

In [ ]:
# 모델 개요 표시 1

print(net)

In [ ]:
# 모델 개요 표시2

summary(net,(100,3,32,32),depth=1)

In [ ]:
# 손실 계산
loss = eval_loss(test_loader2, device, net, criterion)

# 손실 계산 그래프 시각화
g = make_dot(loss, params=dict(net.named_parameters()))
display(g)

## 9.10 결과(CNN)

### 학습

In [ ]:
# CNN 모델 초기화와 학습
torch_seed() # 난수 초기화
net = CNN(n_output, n_hidden).to(device) # 모델 인스턴스 생성
criterion = nn.CrossEntropyLoss() # 손실 함수
lr = 0.01 # 학습률
optimizer = optim.SGD(net.parameters(), lr=lr) # 최적화 함수 : 경사 하강법
num_epochs = 50 # 반복 횟수
history2 = np.zeros((0, 5)) # 평가 결과 기록

# 학습
history2 = fit(net, optimizer, criterion, num_epochs, train_loader2, test_loader2, device, history2)

### 평가

In [ ]:
# 평가

evaluate_history(history2)

In [ ]:
# 처음 50개 데이터 표시

show_images_labels(test_loader2, classes, net, device)